## Data ETL Notebook

**Layer**: Silver

**Domain**: Risk-free

**Action**: Transform RBNZ Yields - Yields as decimals e.g. #.#% becomes 0.0##

In [0]:
%sql
-- Read current records from Silver cleaned and merge Series data (overwrite current version)
CREATE OR REPLACE TABLE workspace.riskfree_silver.003_rbnz_yields_transformed AS
SELECT
    -- keep source columns
    date,
    series_id,
    yield_percent,
    publish_date,
    source_file_name,
    ingestion_timestamp,
    group,
    series,
    unit,

    -- derive term_yr
    CASE
        -- Case 1: If series contains 'days' (e.g. '30 days')
        WHEN lower(series) LIKE '%days%' THEN 
            TRY_CAST(REGEXP_EXTRACT(series, '(\\d+)') AS DOUBLE) / 360

        -- Case 2: If series contains 'year' (e.g. '2 year')
        WHEN lower(series) LIKE '%year%' THEN 
            TRY_CAST(REGEXP_EXTRACT(series, '(\\d+(\\.\\d+)?)') AS DOUBLE)

        -- Case 3: If series contains full date (e.g. '20 September 2025')
        WHEN series RLIKE '\\d{1,2} [A-Za-z]+ \\d{4}' THEN 
            DATEDIFF(TO_DATE(series, 'd MMMM yyyy'), publish_date) / 365.25

        -- Case 4: If series contains partial date (e.g. 'March 2025'), assume day = 15
        WHEN series RLIKE '^[A-Za-z]+ \\d{4}$' THEN 
            DATEDIFF(TO_DATE(CONCAT('15 ', series), 'd MMMM yyyy'), publish_date) / 365.25

        -- Default case
        ELSE 0
    END AS term_yr,

    -- whole number of months, rounded down
    FLOOR(
        CASE
            WHEN lower(series) LIKE '%days%' THEN 
                TRY_CAST(REGEXP_EXTRACT(series, '(\\d+)') AS DOUBLE) / 360

            WHEN lower(series) LIKE '%year%' THEN 
                TRY_CAST(REGEXP_EXTRACT(series, '(\\d+(\\.\\d+)?)') AS DOUBLE)

            WHEN series RLIKE '\\d{1,2} [A-Za-z]+ \\d{4}' THEN 
                DATEDIFF(TO_DATE(series, 'd MMMM yyyy'), publish_date) / 365.25

            WHEN series RLIKE '^[A-Za-z]+ \\d{4}$' THEN 
                DATEDIFF(TO_DATE(CONCAT('15 ', series), 'd MMMM yyyy'), publish_date) / 365.25

            ELSE 0
        END * 12
    ) AS term_mth_whole,

    -- yield decimal
    yield_percent / 100 AS yield_decimal

FROM workspace.riskfree_silver.002_rbnz_yields_enriched
;

-- Confirmation message
SELECT '✅ Silver table created: workspace.riskfree_silver.003_rbnz_yields_transformed' AS message;